## Retrieving bioactivity data for drugs & probes from ChEMBL

In [40]:
df_drugs = pd.read_csv("/Users/bzdrazil/Dropbox/OT/Drug2Target/drug_to_target_unique_drugs.csv")

In [41]:
df_drugs.head()

,drugId
0,CHEMBL1200632
1,CHEMBL1231
2,CHEMBL1233511
3,CHEMBL1637
4,CHEMBL1743017


In [44]:
list = df_drugs['drugId'].tolist()

In [45]:
list

['CHEMBL1200632',
 'CHEMBL1231',
 'CHEMBL1233511',
 'CHEMBL1637',
 'CHEMBL1743017',
 'CHEMBL185885',
 'CHEMBL1949708',
 'CHEMBL2105675',
 'CHEMBL2107826',
 'CHEMBL2109673',
 'CHEMBL2346976',
 'CHEMBL279115',
 'CHEMBL3181832',
 'CHEMBL3545096',
 'CHEMBL3545103',
 'CHEMBL3545145',
 'CHEMBL3545312',
 'CHEMBL363648',
 'CHEMBL3707249',
 'CHEMBL3989766',
 'CHEMBL3989990',
 'CHEMBL4297414',
 'CHEMBL4297785',
 'CHEMBL4302505',
 'CHEMBL4650300',
 'CHEMBL466659',
 'CHEMBL471737',
 'CHEMBL4847050',
 'CHEMBL521851',
 'CHEMBL524',
 'CHEMBL604608',
 'CHEMBL8260',
 'CHEMBL850',
 'CHEMBL852',
 'CHEMBL959',
 'CHEMBL110739',
 'CHEMBL2106966',
 'CHEMBL2107840',
 'CHEMBL4650743',
 'CHEMBL2159741',
 'CHEMBL1090090',
 'CHEMBL1200656',
 'CHEMBL1200879',
 'CHEMBL1200887',
 'CHEMBL1201746',
 'CHEMBL1240705',
 'CHEMBL139367',
 'CHEMBL1585',
 'CHEMBL1721885',
 'CHEMBL1761322',
 'CHEMBL188921',
 'CHEMBL197672',
 'CHEMBL2023898',
 'CHEMBL2103840',
 'CHEMBL2108660',
 'CHEMBL2108866',
 'CHEMBL2108887',
 'CHEMBL21956

In [5]:
pip install cx_oracle --upgrade

Note: you may need to restart the kernel to use updated packages.


In [13]:
import cx_Oracle

#cx_Oracle.init_oracle_client(lib_dir="/Users/bzdrazil/Downloads/instantclient_19_8")  #https://www.oracle.com/in/database/technologies/instant-client/macos-intel-x86-downloads.html; https://stackoverflow.com/questions/56119490/cx-oracle-error-dpi-1047-cannot-locate-a-64-bit-oracle-client-library
cx_Oracle.clientversion() 

import pandas as pd
import sqlalchemy as sa
# import requests
import re
import datetime as dt

workdir = '/Users/bzdrazil/Desktop/'

chempro = 'oracle://{}:{}@ora-vm-089.ebi.ac.uk:1531/?service_name=chempro'.format('', '') #login and password are required
engine_ch = sa.create_engine(chempro)

In [46]:
#chembl_drugs = ['CHEMBL1000', 'CHEMBL100014']

chembl_drugs = list

temp = []
with engine_ch.begin() as conn:
    for v in chembl_drugs:
        q = '''
    
            select md.chembl_id, md.molregno, td.tid, cs.accession, td.pref_name, td.chembl_id as target_chembl_id,  MIN( act.standard_value) as MIN_standard_value, act.standard_units from chembl.molecule_dictionary md
            join chembl.activities act on act.molregno = md.molregno
            join chembl.assays a on a.assay_id = act.assay_id
            join chembl.target_dictionary td on td.tid = a.tid
            join chembl.target_components tc on tc.tid = td.tid
            join chembl.component_sequences cs on cs.component_id = tc.component_id
            where md.chembl_id = '{}'
            and act.standard_value is not NULL
            and act.standard_units is not NULL
            and td.organism = 'Homo sapiens'
            and act.activity_comment is NULL
            and act.standard_relation = '='
            GROUP BY (md.chembl_id, md.molregno, td.tid, td.pref_name, td.chembl_id, cs.accession, act.standard_units, act.standard_relation )

            '''.format(v, v, v)
        temp.append(pd.read_sql((q), conn ))
print(q)


    
            select md.chembl_id, md.molregno, td.tid, cs.accession, td.pref_name, td.chembl_id as target_chembl_id,  MIN( act.standard_value) as MIN_standard_value, act.standard_units from chembl.molecule_dictionary md
            join chembl.activities act on act.molregno = md.molregno
            join chembl.assays a on a.assay_id = act.assay_id
            join chembl.target_dictionary td on td.tid = a.tid
            join chembl.target_components tc on tc.tid = td.tid
            join chembl.component_sequences cs on cs.component_id = tc.component_id
            where md.chembl_id = 'CHEMBL315076'
            and act.standard_value is not NULL
            and act.standard_units is not NULL
            and td.organism = 'Homo sapiens'
            and act.activity_comment is NULL
            and act.standard_relation = '='
            GROUP BY (md.chembl_id, md.molregno, td.tid, td.pref_name, td.chembl_id, cs.accession, act.standard_units, act.standard_relation )

            


In [47]:
df = temp[0]
for i in range(0,len(temp)-1):
    df = df.merge(temp[i+1], how='outer')

/Users/bzdrazil/opt/anaconda3/lib/python3.9/site-packages/pandas/core/reshape/merge.py:1204: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  warnings.warn(


In [48]:
df.head()

,chembl_id,molregno,tid,accession,pref_name,target_chembl_id,min_standard_value,standard_units
0,CHEMBL1231,229058,11570,P17405,Sphingomyelin phosphodiesterase,CHEMBL2760,94.200,%
1,CHEMBL1231,229058,104691,P20309,Muscarinic acetylcholine receptor,CHEMBL2094109,0.600,mg.kg-1
2,CHEMBL1231,229058,104691,P08172,Muscarinic acetylcholine receptor,CHEMBL2094109,0.600,mg.kg-1
3,CHEMBL1231,229058,215,P08912,Muscarinic acetylcholine receptor M5,CHEMBL2035,2.315,nM
4,CHEMBL1231,229058,104691,P08912,Muscarinic acetylcholine receptor,CHEMBL2094109,0.600,mg.kg-1


In [59]:
df_sorted = df.sort_values(by = 'chembl_id', ascending = True)

In [61]:
df_sorted.head()

,chembl_id,molregno,tid,accession,pref_name,target_chembl_id,min_standard_value,standard_units
14498,CHEMBL1000,111185,104067,Q96FL8,Multidrug and toxin extrusion protein 1,CHEMBL1743126,2.00,%
14490,CHEMBL1000,111185,17045,P08684,Cytochrome P450 3A4,CHEMBL340,13.00,%
14489,CHEMBL1000,111185,165,Q12809,HERG,CHEMBL240,30199.52,nM
14488,CHEMBL1000,111185,103947,Q9Y6L6,Solute carrier organic anion transporter famil...,CHEMBL1697668,35.70,%
14493,CHEMBL1000,111185,127,P35367,Histamine H1 receptor,CHEMBL231,5.89,nM


In [62]:
df_sorted.to_csv("/Users/bzdrazil/Dropbox/OT/Drug2Target/drug2target_bioactivities_chembl_33_grouped.csv")

In [49]:
len(df)

64695

In [50]:
df.chembl_id.nunique()

7329

In [76]:
#chembl_drugs = ['CHEMBL1000', 'CHEMBL100014']

chembl_drugs = list

temp = []
with engine_ch.begin() as conn:
    for v in chembl_drugs:
        q = '''
            select md.chembl_id, md.molregno, a.chembl_id as assay_chembl_id, d.src_id, s.src_short_name, d.journal, d.pubmed_id, d.doi, td.tid, cs.accession, td.pref_name, td.chembl_id as target_chembl_id, td.organism, act.standard_value, act.standard_units, act.standard_relation, act.pchembl_value from chembl.molecule_dictionary md
            join chembl.activities act on act.molregno = md.molregno
            join chembl.assays a on a.assay_id = act.assay_id
            join chembl.target_dictionary td on td.tid = a.tid
            join chembl.target_components tc on tc.tid = td.tid
            join chembl.component_sequences cs on cs.component_id = tc.component_id
            join chembl.docs d on d.doc_id = a.doc_id
            join chembl.source s on s.src_id = a.src_id
            where md.chembl_id = '{}'
            and act.standard_value is not NULL
            and act.standard_units is not NULL
            and td.organism = 'Homo sapiens'
            and act.activity_comment is NULL
            '''.format(v, v, v)
        temp.append(pd.read_sql((q), conn ))
print(q)


            select md.chembl_id, md.molregno, a.chembl_id as assay_chembl_id, d.src_id, s.src_short_name, d.journal, d.pubmed_id, d.doi, td.tid, cs.accession, td.pref_name, td.chembl_id as target_chembl_id, td.organism, act.standard_value, act.standard_units, act.standard_relation, act.pchembl_value from chembl.molecule_dictionary md
            join chembl.activities act on act.molregno = md.molregno
            join chembl.assays a on a.assay_id = act.assay_id
            join chembl.target_dictionary td on td.tid = a.tid
            join chembl.target_components tc on tc.tid = td.tid
            join chembl.component_sequences cs on cs.component_id = tc.component_id
            join chembl.docs d on d.doc_id = a.doc_id
            join chembl.source s on s.src_id = a.src_id
            where md.chembl_id = 'CHEMBL315076'
            and act.standard_value is not NULL
            and act.standard_units is not NULL
            and td.organism = 'Homo sapiens'
            and act.acti

In [95]:
len(temp)

12835

In [97]:
df = temp[0]
for i in range(0,len(temp)-1):
    df = pd.concat([temp[i+1]], join='outer')

In [101]:
df = temp[0]
for i in range(0,len(temp)-1):
    df = df.append(temp[i+1])

In [104]:
len(df)

198397

In [105]:
df.head()

,chembl_id,molregno,assay_chembl_id,src_id,src_short_name,journal,pubmed_id,doi,tid,accession,pref_name,target_chembl_id,organism,standard_value,standard_units,standard_relation,pchembl_value
0,CHEMBL1231,229058,CHEMBL1909171,15,DRUGMATRIX,None,NaN,None,47,P08172,Muscarinic acetylcholine receptor M2,CHEMBL211,Homo sapiens,27.000,nM,=,7.57
1,CHEMBL1231,229058,CHEMBL1909171,15,DRUGMATRIX,None,NaN,None,47,P08172,Muscarinic acetylcholine receptor M2,CHEMBL211,Homo sapiens,9.612,nM,=,8.02
2,CHEMBL1231,229058,CHEMBL891646,1,LITERATURE,Bioorg Med Chem Lett,17188867.0,10.1016/j.bmcl.2006.11.058,47,P08172,Muscarinic acetylcholine receptor M2,CHEMBL211,Homo sapiens,8.100,nM,=,8.09
3,CHEMBL1231,229058,CHEMBL863986,1,LITERATURE,Bioorg Med Chem Lett,16275087.0,10.1016/j.bmcl.2005.09.079,47,P08172,Muscarinic acetylcholine receptor M2,CHEMBL211,Homo sapiens,16.000,nM,=,7.8
4,CHEMBL1231,229058,CHEMBL2045510,1,LITERATURE,J Med Chem,22243489.0,10.1021/jm2013216,47,P08172,Muscarinic acetylcholine receptor M2,CHEMBL211,Homo sapiens,11.750,nM,=,7.93


In [106]:
df_sorted_2 = df.sort_values(by = 'chembl_id', ascending = True)

In [107]:
df_sorted_2.to_csv("/Users/bzdrazil/Dropbox/OT/Drug2Target/drug2target_bioactivities_chembl_33.csv")